<a href="https://colab.research.google.com/github/HugoStigletz/Data-Analytics-Portfolio/blob/main/scrapping_with_mechanicalsoup_sqlite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# https://en.wikipedia.org/wiki/Comparison_of_Linux_distributions
import mechanicalsoup
import pandas as pd
import sqlite3

In [1]:
pip install mechanicalsoup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 13.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://en.wikipedia.org/wiki/Comparison_of_Linux_distributions")

<Response [200]>

In [10]:
#extract table headers
th = browser.page.find_all('th', attrs={"class": "table-rh"})
distribution = [value.text.replace("\n", "") for value in th]
distribution = distribution [:98]
print(distribution)
# print(distribution.index("Zorin OS")) (Was used to locate the last index of the table we needed rather than pulling all tables from web)

['AlmaLinux', 'Alpine Linux', 'ALT Linux', 'antiX', 'ArchBang', 'Arch Linux', 'BLAG', 'Bodhi Linux', 'Canaima', 'CentOS', 'Chakra', 'Chrome OS', 'Clear Linux OS', 'ClearOS', 'CrunchBang Linux', 'Damn Small Linux', 'Debian', 'Debian Edu', 'Devuan', 'Deepin', 'Dragora GNU/Linux-Libre', 'dyne:bolic', 'Elementary OS', 'ELinOS', 'Emdebian Grip', 'EndeavourOS', 'Fedora Linux', 'Freespire', 'Gentoo Linux', 'Guix System', 'gNewSense', 'Grml', 'Hyperbola GNU/Linux-libre', 'Kali Linux', 'Knoppix', 'Kodibuntu', 'Korora', 'LibreCMC', 'Linspire', 'Linux Mint', 'Linux Lite', 'Mageia', 'Mandriva Linux', 'Manjaro Linux', 'MEPIS', 'MIRACLE LINUX', 'Musix GNU+Linux', 'Netrunner', 'NixOS', 'Novell Open Enterprise Server', 'OpenELEC', 'openSUSE', 'OpenWrt', 'OpenMandriva Lx', 'Oracle Linux', 'Parabola GNU/Linux-libre', 'Pardus', 'Parrot OS', 'Parsix', 'Parted Magic', 'PCLinuxOS', 'Pop! OS', 'Pentoo', 'Porteus', 'Puppy Linux', 'PureOS', 'Qubes OS', 'Red Hat Enterprise Linux (RHEL)', 'Red Hat Linux', 'Rocks

In [12]:
#extract table data
td = browser.page.find_all("td")
columns = [value.text.replace("\n", "") for value in td]
columns = columns[6:1084]
print(columns)
#print(columns.index("AlmaLinux Foundation")) (This is where our code starts)
#print(columns.index("Binary blobs")) (This is right after the last cell of the table we need because the last word "ACTIVE" is too generic to locate through search)

['AlmaLinux Foundation', 'AlmaLinux Foundation', '2021', '9.0', '10 years[2]', '2022-05-26', 'X', 'Red Hat Enterprise Linux (RHEL)', 'server, workstation', 'None', 'Active', 'Alpine Linux Team', 'Alpine Linux Team', '2006', '3.16[3]', '?', '2022-05-23', 'X', 'LEAF Project', 'security, lightweight, general', 'None', 'Active', 'ALT Linux Team', 'ALT Linux Team, ALT Linux LLC', '2001', '10.0[4]', '?', '2021-12-16', 'X', 'Mandrake Linux', 'general, school', 'None', 'Active', 'Anticapitalista', 'Anticapitalista', '2007', '19.4[5]', '?', '2021-05-21', 'X', 'Debian, MEPIS', 'old computers', 'None', 'Active', 'Willensky Aristide', 'Stan McLaren', '2011', 'Rolling', 'Rolling', '?', 'X', 'Arch Linux (UKM Edition)', 'general', 'None', 'Active', 'Judd Vinet', 'dev team', '2002', 'Rolling', 'Rolling', 'Rolling', 'X', 'Independent, inspired from CRUX', 'general', 'None', 'Active', 'Brixton Linux Action Group', 'Brixton Linux Action Group', '2002', '140k', '?', '2011-05-04', '100% Free: GNU FSDG[6]',

In [16]:
column_names = ["Founder", 
                "Maintainer", 
                "Initial_Release_Year", 
                "Current_Stable_Version", 
                "Security_Updates", 
                "Release_Date", 
                "System_Distribution_Commitment", 
                "Forked_From", 
                "Target_Audience", 
                "Cost", 
                "Status"]

dictionary = {"Distribution": distribution}

for idx, key in enumerate(column_names):
  dictionary[key] = columns[idx:][::11]

df = pd.DataFrame(data = dictionary)
print(df.tail())

    Distribution                  Founder            Maintainer  \
93  Webconverger               Kai Hendry  Webconverger Limited   
94       Xandros     Xandros Incorporated  Xandros Incorporated   
95       Zentyal        eBox Technologies     eBox Technologies   
96       Zenwalk  Jean-Philippe Guillemin              dev team   
97      Zorin OS              Zorin Group           Zorin Group   

   Initial_Release_Year Current_Stable_Version Security_Updates Release_Date  \
93                 2007                   35.0                ?   2016-05-19   
94                 2001                      ?                ?   2007-07-26   
95                 2005                6.2[82]                ?   2020-06-04   
96                 2004            Rolling[83]                ?   2020-12-10   
97                 2009            OS 16.1[84]                ?   2022-03-10   

   System_Distribution_Commitment     Forked_From  \
93                              X          Debian   
94        

In [ ]:
#select every 11th item
columns[0:][::11]
columns[1:][::11]
columns[2:][::11]
columns[3:][::11]
columns[4:][::11]
columns[5:][::11]
columns[6:][::11]
columns[7:][::11]
columns[8:][::11]
columns[9:][::11]
columns[10:][::11]

In [20]:
# insert data into a database
connection = sqlite3.connect("linux_distro.db")
cursor = connection.cursor()

cursor.execute("create table linux (Distribution, " + ",".join(column_names)+ ")")
for i in range(len(df)):
    cursor.execute("insert into linux values (?,?,?,?,?,?,?,?,?,?,?,?)", df.iloc[i])

connection.commit()

connection.close()